## `Learner` for Multi-Label Classifier Fine-Tuning (Frozen)

In [ ]:
dls_clas = torch.load(path_model/'dls_clas_full_64.pkl')
# dls_clas = torch.load(path_model/'dls_clas_sample_64.pkl')

### Architecture Creation:

To create an architecture that we can use to build a `model` we need to unwrap fastai's `get_text_classifier`. Let's do that: 

In [ ]:
# model = get_text_classifier(AWD_LSTM,
#                             len(vocab),
#                             n_out,
#                             seq_len=seq_len,
#                             config=config,
#                             y_range=y_range,
#                             drop_mult=drop_mult,
#                             lin_ftrs=lin_ftrs,
#                             max_len=max_len)

The following is a dictionary where keys are the well known (possibly pretarianed) NLP architectures and values are the configurations, urls (where we can download the pretrained weights from) and other info we need to create that architecture:

In [ ]:
_model_meta = {
    AWD_LSTM: {'hid_name':'emb_sz', 'url':URLs.WT103_FWD, 'url_bwd':URLs.WT103_BWD,'config_lm':awd_lstm_lm_config, 'split_lm': awd_lstm_lm_split,            
                'config_clas':awd_lstm_clas_config, 'split_clas': awd_lstm_clas_split
              }
    }

Let's define an architecture:

In [ ]:
arch = AWD_LSTM
arch

fastai.text.models.awdlstm.AWD_LSTM

Let's get the value (information) of that architecture:

In [ ]:
meta = _model_meta[arch]
meta

{'hid_name': 'emb_sz',
 'url': 'https://s3.amazonaws.com/fast-ai-modelzoo/wt103-fwd.tgz',
 'url_bwd': 'https://s3.amazonaws.com/fast-ai-modelzoo/wt103-bwd.tgz',
 'config_lm': {'emb_sz': 400,
  'n_hid': 1152,
  'n_layers': 3,
  'pad_token': 1,
  'bidir': False,
  'output_p': 0.1,
  'hidden_p': 0.15,
  'input_p': 0.25,
  'embed_p': 0.02,
  'weight_p': 0.2,
  'tie_weights': True,
  'out_bias': True},
 'split_lm': <function fastai.text.models.awdlstm.awd_lstm_lm_split(model)>,
 'config_clas': {'emb_sz': 400,
  'n_hid': 1152,
  'n_layers': 3,
  'pad_token': 1,
  'bidir': False,
  'output_p': 0.4,
  'hidden_p': 0.3,
  'input_p': 0.4,
  'embed_p': 0.05,
  'weight_p': 0.5},
 'split_clas': <function fastai.text.models.awdlstm.awd_lstm_clas_split(model)>}

Let's now get the default configuration required to build that architecture:

In [ ]:
config = None
config = ifnone(config, meta['config_clas']).copy()
config

{'emb_sz': 400,
 'n_hid': 1152,
 'n_layers': 3,
 'pad_token': 1,
 'bidir': False,
 'output_p': 0.4,
 'hidden_p': 0.3,
 'input_p': 0.4,
 'embed_p': 0.05,
 'weight_p': 0.5}

Alternatively, we could have just grabbed the configuration provided by a fastai convenienece dictionary called `awd_lstm_clas_config`:

In [ ]:
# config = awd_lstm_clas_config.copy()
# config

**Dropout**:  
As we can see the the default configuration has some default dropout probabilities. We can use a multiplier to multiply with each of these probabilities:

In [ ]:
drop_mult = 0.1
for k in config.keys():
    if k.endswith('_p'): config[k] *= drop_mult

NOT SURE (Find out later!) 

Note: In fastai `ps` was $0.1$, we made it $0.01$ (because in extreme multi-label classification we want to drop out less activations, but need to investigate this later!)

In [ ]:
lin_ftrs, ps = None, None
if lin_ftrs is None: lin_ftrs = [50]
if ps is None: ps = [0.1] * len(lin_ftrs)
ps, lin_ftrs

([0.1], [50])

Let's find out the total number of classes from dls (becuase we need this info to build the architecture)

In [ ]:
n_out = get_c(dls_clas)
n_out

8922

The embedding size as per the default config is:

In [ ]:
emb_sz = config.get('emb_sz')
emb_sz

400

The decoder for text classification will be a `PoolingLinearClassifier` with two linear layers:
- the first layer will have number of input features 1200 and number of output features 50
- the second layer will have number of input features 50 and number of output features equal to `n_out`

Let's make these layer configuarion in a list called `layers`

In [ ]:
layers = [emb_sz * 3] + lin_ftrs + [n_out]
layers

[1200, 50, 8922]

---

**IMPORTANT**: 

In fastai the number of output features coming out was 400. Then we did a `masked_concat_pool` so the final output fed to the `PoolingLinearClassifier` was 1200. In order to implement **attention** we concat a context of 400 to those previous 1200 features. So the total number output features with attention is 1200+400=1600. Hence the change below:

If using attention:

In [ ]:
layers_attn = [n_out * (emb_sz * 3 + emb_sz) ] + lin_ftrs + [n_out]
layers_attn

[14275200, 50, 8922]

---

In [ ]:
print(ps)
ps = [config.pop('output_p')] + ps
print(ps)

[0.1]
[0.04000000000000001, 0.1]


In [ ]:
init = config.pop('init') if 'init' in config else None

Now we have reached a point where we can make our Encoder (which in this case will be an AWD_LSTM that is the architecture that we are using). To make the `AWD_LSTM` module we need to pass the `vocab_sz` and the `config` dictionary (after peeling off `output_p` and `init`):

In [ ]:
vocab = dls_clas.vocab[0]
len(vocab)

60008

In [ ]:
print(f"{arch = }, \n\n {config = }")

arch = <class 'fastai.text.models.awdlstm.AWD_LSTM'>, 

 config = {'emb_sz': 400, 'n_hid': 1152, 'n_layers': 3, 'pad_token': 1, 'bidir': False, 'hidden_p': 0.03, 'input_p': 0.04000000000000001, 'embed_p': 0.005000000000000001, 'weight_p': 0.05}


Let's take a look at how our AWD_LSTM model looks:

In [ ]:
arch

fastai.text.models.awdlstm.AWD_LSTM

In [ ]:
# config.update({'bidir': True})
# config

In [ ]:
arch(vocab_sz=len(vocab), **config)

AWD_LSTM(
  (encoder): Embedding(60008, 400, padding_idx=1)
  (encoder_dp): EmbeddingDropout(
    (emb): Embedding(60008, 400, padding_idx=1)
  )
  (rnns): ModuleList(
    (0): WeightDropout(
      (module): LSTM(400, 1152, batch_first=True)
    )
    (1): WeightDropout(
      (module): LSTM(1152, 1152, batch_first=True)
    )
    (2): WeightDropout(
      (module): LSTM(1152, 400, batch_first=True)
    )
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
    (2): RNNDropout()
  )
)

**An important thing (of immense practical importance) to note:**  
Since we do not have infinite amount of GPU memory so we need to wrap our AWD_LSTM module inside a `SentenceEncoder`.  
Need to elaborate further (later on!)

fastai's `SentenceEncoder` takes the following positional and keyword arguments:
- `bptt` (this is chunk size, mostly `seq_len`, the text document gets broken into)
- `module` (this the module that we want to wrap a `SentenceEncoder` around)
- `pad_idx` (has a default value of 1 everywhere in fastai)
- `max_len` (has a default value 72 * 20)

We don't really need to write our own `SentenEncoder` we can just use fastai's. But it is exteremely important to understand this source code. So for the sake of completion we include it below:

**IMPORTANT:** To make sure we look at the hidden state for each of the token in the entire document (meaning we want to be looking at `bs,seq_len,nh` where `seq_len` is the entire document length and not just `max_len`) before classifying, we need to let `SentenceEncoder` work with its default `max_len` (instead of passing something like $72*20$ which is what we would do for sentiment analysis)... [Elaborate this later explaining the difference between extreme multilabel classification and sentiment analysis...]

In [ ]:
seq_len = 72
# encoder = OurSentenceEncoder(seq_len, arch(vocab_sz=len(vocab), **config), pad_idx=1, max_len=seq_len*20)
encoder = SentenceEncoder(seq_len, arch(vocab_sz=len(vocab), **config), pad_idx=1, max_len=seq_len*20)
encoder

SentenceEncoder(
  (module): AWD_LSTM(
    (encoder): Embedding(60008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
)

So now have the encoder and ready to make the decoder. The decoder in this case would be a `PoolingLinearClassifier`. We will make it using the following information:

In [ ]:
y_range = None
layers, ps, seq_len

([1200, 50, 8922], [0.04000000000000001, 0.1], 72)

---

If using attention:

In [ ]:
layers_attn, ps, seq_len

([14275200, 50, 8922], [0.04000000000000001, 0.1], 72)

---

Eventhough now we can just use fastai's `PoolingLinearClassifier`, but later on we will need to modify this module (to incorporate *attention*). So for the purpose of customizability let's copy the source code from fastai and (shamelessly) call it `OurPoolingLinearClassifier`:

In [ ]:
class OurPoolingLinearClassifier(Module):
    def __init__(self, dims, ps, bptt, y_range=None):
        if len(ps) != len(dims)-1: raise ValueError("Number of layers and dropout values do not match.")
        acts = [nn.ReLU(inplace=True)] * (len(dims) - 2) + [None]
        layers = [LinBnDrop(i, o, p=p, act=a) for i,o,p,a in zip(dims[:-1], dims[1:], ps, acts)]
        if y_range is not None: layers.append(SigmoidRange(*y_range))
        self.layers = nn.Sequential(*layers)
        self.bptt = bptt

    def forward(self, input):
        out, mask = input
        x = masked_concat_pool(out, mask, self.bptt)
        x = self.layers(x)
        return x, out, out

#### Breaking down the `OurPoolingLinearClassifier.__init__`:

Note that in the `__init__` while creating `OurPoolingLinearClassifier` `dims` is `layers`

In [ ]:
dims = layers
print(f"{dims = }")

dims = [1200, 50, 6594]


In [ ]:
print(f"{ps = }")

ps = [0.04000000000000001, 0.1]


Also note that `bptt` is `seq_len`

In [ ]:
bptt = seq_len
print(f"{bptt = }")

bptt = 72


In [ ]:
y_range = None

In [ ]:
if len(ps) != len(dims) - 1: raise ValueError("Number of layers and dopout values do not match.")

In [ ]:
acts = [nn.ReLU(inplace=True)] * (len(dims) - 2) + [None]
acts

[ReLU(inplace=True), None]

In [ ]:
for i, o, p, a in zip(dims[:-1], dims[1:], ps, acts):
    print(f"{i = }, {o = }, {p = }, {a = }")

i = 1200, o = 50, p = 0.04000000000000001, a = ReLU(inplace=True)
i = 50, o = 6594, p = 0.1, a = None


In [ ]:
layers = [LinBnDrop(i, o, p=p, act=a) for i, o, p, a in zip(dims[:-1], dims[1:], ps, acts)]
layers

[LinBnDrop(
   (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1): Dropout(p=0.04000000000000001, inplace=False)
   (2): Linear(in_features=1200, out_features=50, bias=False)
   (3): ReLU(inplace=True)
 ),
 LinBnDrop(
   (0): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1): Dropout(p=0.1, inplace=False)
   (2): Linear(in_features=50, out_features=6594, bias=False)
 )]

---

CAUTION: Work in Progress!

#### Attempt #1: Implementing [CAML](https://arxiv.org/pdf/1802.05695.pdf)

In [ ]:
class CAML1(Module):
    def __init__(self, dims, ps, bptt):
        layers = [LinBnDrop(dims[0], 50, p=ps, act=nn.ReLU(inplace=True))]
        layers.append(LinBnDrop(50, n_out, p=0.1, act=None))
        self.layers = nn.Sequential(*layers)
        self.bptt = bptt

    def forward(self, input):
        out, mask = input
        x = masked_concat_pool(out, mask, self.bptt)
        x = self.layers(x)
        return x, out, out

In [ ]:
CAML1(dims=[1200, 6594], ps=0.04, bptt=seq_len)

CAML1(
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.04, inplace=False)
      (2): Linear(in_features=1200, out_features=50, bias=False)
      (3): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): Linear(in_features=50, out_features=6594, bias=False)
    )
  )
)

In [ ]:
class CAML2(Module):
    def __init__(self, dims, ps, bptt):
        self.layer = LinBnDrop(dims[0], dims[1], p=ps, act=None)
        self.bptt = bptt

    def forward(self, input):
        out, mask = input
        x = masked_concat_pool(out, mask, self.bptt)
        x = self.layer(x)
        return x, out, out

In [ ]:
decoder = CAML2(dims=[1200, 8922], ps=0.04, bptt=seq_len)

Note: Also try `CAML2` w/o dropouts and batch normalization (Verify this, but as far as what I found it does not work well as compared to /w batch normalization)

In [ ]:
class LinBn2Drop(nn.Sequential):
    "Module grouping `BatchNorm2d`, `Dropout` and a `Linear` layer with just one output feature"
    def __init__(self, n_lbs, n_fts, bn=True, p=0., act=None, lin_first=False):
        layers = [BatchNorm(n_lbs, ndim=1)] if bn else []
        if p != 0: layers.append(nn.Dropout(p))
        lin = [nn.Linear(n_fts, 1, bias=not bn)]
        if act is not None: lin.append(act)
        layers = lin+layers if lin_first else layers+lin
        super().__init__(*layers)

In [ ]:
class CAML3(Module):
    def __init__(self, ps, bptt):
        self.layers = LinBn2Drop(n_out, emb_sz, p=ps, act=None) # deb
        self.bptt = bptt
        self.emb_label = nn.Embedding(n_out, emb_sz) # deb
        

    def forward(self, input):
        out, _ = input
        # breakpoint()
        # x = masked_concat_pool(out, mask, self.bptt)
        attn_wgts = out @ self.emb_label.weight.transpose(0, 1) # deb
        attn_wgts = F.softmax(attn_wgts, 1) # deb
        ctx = attn_wgts.transpose(1,2) @ out # deb
        
        x = self.layers(ctx)
        x = x.view(x.shape[0], x.shape[1])
        return x, out, out

In [ ]:
CAML3(ps=0.04, bptt=seq_len)

CAML3(
  (layers): LinBn2Drop(
    (0): BatchNorm1d(6594, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.04, inplace=False)
    (2): Linear(in_features=400, out_features=1, bias=False)
  )
  (emb_label): Embedding(6594, 400)
)

---

In [ ]:
out = torch.randn((128, 1406, 400)).cuda()
out.shape, out.device

(torch.Size([128, 1406, 400]), device(type='cuda', index=0))

In [ ]:
attn_wgts = torch.randn((128, 1406, 6594)).cuda()
attn_wgts.shape, attn_wgts.device

(torch.Size([128, 1406, 6594]), device(type='cuda', index=0))

In [ ]:
#out[:, :, None].shape

In [ ]:
attn_wgts.transpose(1,2).shape

torch.Size([128, 6594, 1406])

In [ ]:
ctx = attn_wgts.transpose(1,2) @ out

In [ ]:
ctx.shape

torch.Size([128, 6594, 400])

In [ ]:
test_model = CAML3(ps=0.04, bptt=seq_len).cuda()

In [ ]:
res, *_ = test_model((out, None))

In [ ]:
res.shape

torch.Size([128, 6594])

---

In [ ]:
class CAML4(Module):
    def __init__(self, dims, ps, bptt):
        self.layer = LinBnDrop(dims[0], dims[1], p=ps, act=None)
        self.bptt = bptt
#         self.emb_label = nn.Embedding(n_out, emb_sz) # deb
#         self.linear_final = nn.Linear(emb_sz) 

    def forward(self, input):
        out, mask = input
        x = masked_concat_pool(out, mask, self.bptt)
#         attn_wgts = x @ self.emb_label.weight.transpose(0, 1) # deb
#         attn_wgts = F.softmax(attn_wgts, 1) # deb
#         ctx = out[:, :, None] * attn_wgts[..., None] # deb
#         ctx = ctx.sum(1)
        x = self.layer(x)
        return x, out, out

In [ ]:
decoder = CAML3(ps=0.04, bptt=seq_len)
decoder

CAML3(
  (layers): LinBn2Drop(
    (0): BatchNorm1d(6594, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.04, inplace=False)
    (2): Linear(in_features=400, out_features=1, bias=False)
  )
  (emb_label): Embedding(6594, 400)
)

#### Attempt #2

In [ ]:
class OurPoolingAttentionClassifier(Module):
    def __init__(self, dims, ps, bptt, y_range=None):
        if len(ps) != len(dims)-1: raise ValueError("Number of layers and dropout values do not match.")
        acts = [nn.ReLU(inplace=True)] * (len(dims) - 2) + [None]
        layers = [LinBnDrop(i, o, p=p, act=a) for i,o,p,a in zip(dims[:-1], dims[1:], ps, acts)]
        if y_range is not None: layers.append(SigmoidRange(*y_range))
        self.layers = nn.Sequential(*layers)
        self.bptt = bptt
        
        # new
        label_emb_sz=49
        self.emb_label = nn.Embedding(n_out, emb_sz)
        self.lin = nn.Linear(emb_sz, emb_sz)
        self.lin_for_tok_red = nn.Linear(seq_len*20, 50)
        self.lin_for_rep_compress = nn.Linear(emb_sz, label_emb_sz)
        self.lin_for_rep_decompress = nn.Linear(label_emb_sz, emb_sz)
        self.V = self._init_param(label_emb_sz)
        # new
    
    def forward(self, input):
        out, mask = input
        x = masked_concat_pool(out, mask, self.bptt)
        
        # new
        num_tok = out.shape[1]
        out = F.pad(out, (0,0,0,seq_len*20-num_tok))
        bs = out.shape[0]
        label_indices = torch.arange(n_out, device=out.device)
        labels = label_indices.repeat(bs, 1)
        after_grabbing_label_embedding = self.emb_label(labels)
        after_first_matmul = self.lin(after_grabbing_label_embedding)
        
        out = self.lin_for_rep_compress(out)
        after_first_matmul = self.lin_for_rep_compress(after_first_matmul)
        
        out = out.permute(0,2,1).contiguous()
        out = self.lin_for_tok_red(out)
        out = out.permute(0,2,1).contiguous()
        after_nonlinearity = torch.tanh(out[:, :, None] + after_first_matmul[:,None])
        attn_wgts = (after_nonlinearity @ self.V)
        ctx = (out[:, :, None] * attn_wgts[..., None])
        ctx = ctx.sum(1)
        
        ctx = self.lin_for_rep_decompress(torch.relu(ctx))
        
        x = x[:, None]
        x = x.repeat(1, n_out, 1)
        x = torch.cat((x, ctx), dim=-1)
        x = x.view(x.shape[0], -1)
        # new
        
        x = self.layers(x)
        return x, out, out
    
    def _init_param(self, *sz): 
        return nn.Parameter(torch.randn(sz)/math.sqrt(sz[0]))

CAUTION ENDS

---

#### Creating the decoder:

In [ ]:
decoder = OurPoolingLinearClassifier(layers, ps, bptt=seq_len, y_range=y_range)
decoder

---

If using attention:

In [ ]:
decoder = OurPoolingAttentionClassifier(layers_attn, ps, bptt=seq_len, y_range=y_range)
decoder

OurPoolingAttentionClassifier(
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(10550400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.04000000000000001, inplace=False)
      (2): Linear(in_features=10550400, out_features=50, bias=False)
      (3): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): Linear(in_features=50, out_features=6594, bias=False)
    )
  )
  (emb_label): Embedding(6594, 400)
  (lin): Linear(in_features=400, out_features=400, bias=True)
  (lin_for_tok_red): Linear(in_features=1440, out_features=50, bias=True)
  (lin_for_rep_compress): Linear(in_features=400, out_features=49, bias=True)
  (lin_for_rep_decompress): Linear(in_features=49, out_features=400, bias=True)
)

---

#### Creating the model:

The last thing that we need to do in order to create our architecture module is stack the `encoder` and the `decoder` using `SequentialRNN` module:

In [ ]:
model = SequentialRNN(encoder, decoder)
model

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60008, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60008, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): CAML2(
    (layer): LinBnDrop(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.04, inplace=False)
      (2): Linear(in_features=1200, out_features=8922, bias=False)
    )
  )
)

In [ ]:
if init is not None: model = model.apply(init)

At this point we are done creating the `model` (that is have replicated every step inside `get_text_classifier`). Next, let's move on to `text_classifier_learner`

---

#### Scratchpad: (Build the pieces to implement attention)

In [ ]:
x, y = dls_clas.one_batch()
x.shape, y.shape, x.device, y.device

(torch.Size([128, 18398]),
 torch.Size([128, 6594]),
 device(type='cuda', index=0),
 device(type='cuda', index=0))

In [ ]:
encoder, decoder = model[0], model[1]
encoder, decoder

(SentenceEncoder(
   (module): AWD_LSTM(
     (encoder): Embedding(60008, 400, padding_idx=1)
     (encoder_dp): EmbeddingDropout(
       (emb): Embedding(60008, 400, padding_idx=1)
     )
     (rnns): ModuleList(
       (0): WeightDropout(
         (module): LSTM(400, 1152, batch_first=True)
       )
       (1): WeightDropout(
         (module): LSTM(1152, 1152, batch_first=True)
       )
       (2): WeightDropout(
         (module): LSTM(1152, 400, batch_first=True)
       )
     )
     (input_dp): RNNDropout()
     (hidden_dps): ModuleList(
       (0): RNNDropout()
       (1): RNNDropout()
       (2): RNNDropout()
     )
   )
 ),
 OurPoolingAttentionClassifier(
   (layers): Sequential(
     (0): LinBnDrop(
       (0): BatchNorm1d(10550400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (1): Dropout(p=0.04000000000000001, inplace=False)
       (2): Linear(in_features=10550400, out_features=50, bias=False)
       (3): ReLU(inplace=True)
     )
     (1): LinBnDr

In [ ]:
encoder, decoder = encoder.cuda(), decoder.cuda()

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
out, mask = encoder(x[:2])
out.shape, mask.shape

(torch.Size([2, 1406, 400]), torch.Size([2, 1406]))

---

After getting the output from the `encoder` we will do a masked concat pooling as usual:

In [ ]:
# out_concat_pool = masked_concat_pool(out, mask, bptt=seq_len)
# out_concat_pool.shape

---

Testing the decoder which is `OurPoolingAttentionCalssifier`: 

In [ ]:
# import gc
# out, mask, preds, model, encoder, decoder, x, y, dls_clas, learn, m, out_concat_pool = None, None, None, None, None, None, None, None, None, None, None, None
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
preds, _, _ = decoder((out, mask))

In [ ]:
preds.shape

torch.Size([2, 6594])

---

In [ ]:
# var = locals().get('out_concat_pool', 'Shit!!!')
# print(var)

In [ ]:
# x, y, encoder, dls_clas, out, mask, ind_var = None, None, None, None, None, None, None
# import gc; gc.collect()

---

In [ ]:
bs, emb_sz, n_out = 128, 400, get_c(dls_clas)
bs, emb_sz, n_out

(128, 400, 6594)

`out` is the memory:

In [ ]:
set_bg('yellow')
out = torch.randn(bs, 1406, emb_sz, dtype=torch.float)
out.shape

torch.Size([128, 1406, 400])

In [ ]:
# del out

- The `out` is the memory: `(bs, sl, nh)`
- meaning of the dimensions:
    * For each of the 128 documents we are focussing on the last 1406 tokens each having a hidden vector of length 400
- Let us say before predicting a label we want to know which part of the memory we want to pay attention to for that particular label
- We need to compute the attention weights for each of the `sl` many `(bs,nh)`
- The shape of the attention weights would be `bs,sl`
---
- To compute the attention weights we will use a mini "neural network"
- **What would be the input to this mini-nn? The input to this neural network would be that particular label embedding that we want to predict**


In [ ]:
set_bg('yellow')
n_labels = n_out

In [ ]:
set_bg('yellow')
label_emb_sz = 49
emb_label = nn.Embedding(n_labels, emb_sz)

In [ ]:
emb_label.weight, emb_label.weight.shape

(Parameter containing:
 tensor([[ 1.0915, -0.2077, -0.4202,  ..., -0.1955, -0.0613, -0.1809],
         [ 0.6431, -0.7658, -0.9118,  ...,  0.7211, -0.7179,  0.5732],
         [-0.0820, -0.8173, -0.0974,  ..., -1.7775, -1.6814, -2.2647],
         ...,
         [ 0.3422, -0.4851, -0.4024,  ...,  0.2219,  0.8045,  0.0061],
         [-1.0910, -0.1145,  2.0704,  ..., -0.0486,  0.2683, -0.0494],
         [ 1.3110, -0.1406,  1.5179,  ..., -2.2415,  0.9447,  0.8938]], requires_grad=True),
 torch.Size([6594, 400]))

In [ ]:
emb_label.weight.transpose(0,1)

tensor([[ 1.0915,  0.6431, -0.0820,  ...,  0.3422, -1.0910,  1.3110],
        [-0.2077, -0.7658, -0.8173,  ..., -0.4851, -0.1145, -0.1406],
        [-0.4202, -0.9118, -0.0974,  ..., -0.4024,  2.0704,  1.5179],
        ...,
        [-0.1955,  0.7211, -1.7775,  ...,  0.2219, -0.0486, -2.2415],
        [-0.0613, -0.7179, -1.6814,  ...,  0.8045,  0.2683,  0.9447],
        [-0.1809,  0.5732, -2.2647,  ...,  0.0061, -0.0494,  0.8938]], grad_fn=<TransposeBackward0>)

In [ ]:
emb_label.weight.transpose(0,1).shape

torch.Size([400, 6594])

In [ ]:
emb_label.weight.data = emb_label.weight.type(dtype=torch.float)

In [ ]:
emb_label.weight.type()

'torch.FloatTensor'

In [ ]:
# emb_label.__dict__
# [o for o in dir(emb_label) if not o.startswith("__")]

In [ ]:
emb_label

Embedding(6594, 400)

In [ ]:
# x = torch.randint(0,5,(10,5))
# x.shape, x

In [ ]:
# emb_label(x).shape

---

In [ ]:
# torch.arange(5)[:,None].shape

In [ ]:
# torch.ones(5, bs).shape

In [ ]:
# labels = torch.ones(bs, 5, dtype=torch.int64) * torch.arange(5)
# labels.shape, labels.ndim

In [ ]:
set_bg('yellow')
label_indices = torch.arange(n_labels)
labels = label_indices.repeat(128, 1)
labels.shape

torch.Size([128, 6594])

Note: In `labels`,
- 0th axis is bs
- 1st axis is label index

In [ ]:
# labels[:10, :]
labels[:10]

tensor([[   0,    1,    2,  ..., 6591, 6592, 6593],
        [   0,    1,    2,  ..., 6591, 6592, 6593],
        [   0,    1,    2,  ..., 6591, 6592, 6593],
        ...,
        [   0,    1,    2,  ..., 6591, 6592, 6593],
        [   0,    1,    2,  ..., 6591, 6592, 6593],
        [   0,    1,    2,  ..., 6591, 6592, 6593]])

In [ ]:
# a_label = labels[:,0]
# a_label.shape

---

In [ ]:
set_bg('yellow')
after_grabbing_label_embedding = emb_label(labels)
after_grabbing_label_embedding.shape

torch.Size([128, 6594, 400])

In [ ]:
# import sys
# sys.getsizeof(after_grabbing_label_embedding)

---

In [ ]:
# a = torch.randint(0, 5, size=(3,2,4))
# b = torch.randint(0, 5, size=(4,5))

In [ ]:
# a, b

In [ ]:
# a @ b

In [ ]:
# (a@b).shape

In [ ]:
# (a@b).permute(1,0,2).contiguous()

---

*Step 0 of mini-nn:* `after_grabbing_label_embedding` is the input to the mini-nn which will answer the following question:   

**Which part of the memory `out` should I pay attention before making a prediction about a label**

*Step 1 of mini-nn: Doing the first matrix multiply:*

In [ ]:
set_bg('yellow')
lin = nn.Linear(emb_sz, emb_sz)
lin

Linear(in_features=400, out_features=400, bias=True)

In [ ]:
lin.weight.data = lin.weight.type(dtype=torch.float)

In [ ]:
set_bg('yellow')
print(after_grabbing_label_embedding.shape)
after_first_matmul = lin(after_grabbing_label_embedding)
after_first_matmul.shape

torch.Size([128, 6594, 400])


torch.Size([128, 6594, 400])

At this point the axes mean the following:
- 0th axis is the bs
- 1st axis is label
- 2nd axis is emb_sz

In [ ]:
# after_first_matmul = after_first_matmul.permute(1,0,2).contiguous()
# after_first_matmul.shape

Think of it this way:  
In each of the 128 documents, each of the 6594 labels has a representation vector of length 400.

*Step 2 of mini-nn: Applying non-linearity:*

But before applying a non-linearity we want to add the `after_first_matmul` to the entire memory `out`

In [ ]:
out.shape

torch.Size([128, 1406, 400])

To this memory we will add a singleton dimension at the 2nd position (think of this as adding a label specific dimension to our memory)

In [ ]:
print(f"{out[:, :, None].shape = }")

out[:, :, None].shape = torch.Size([128, 1406, 1, 400])


We want to literally add to our memory the `after_first_matmul`. So to line things up perfectly we add a singleton dimension at the 1st position in `after_first_matmul` (think of this as the token specific dimension)

In [ ]:
print(f"{after_first_matmul[:,None].shape = }")

after_first_matmul[:,None].shape = torch.Size([128, 1, 6594, 400])


In [ ]:
# out.type(torch.half), after_first_matmul.type(torch.half)

In [ ]:
# test = after_nonlinearity = torch.tanh(out[:, :50, None] + after_first_matmul[:,None])

---

At this point we are faced with a realistic issue:  
We cannot afford to 
- consider all the 1406 tokens in our memory (stored in `out`), and 
- have those tokens have a representation length of 400.   
Solution: (Like always) We we will have to do a matrix multiplication to preform compression:

In [ ]:
out.shape

torch.Size([128, 1406, 400])

In [ ]:
set_bg('yellow')
lin_for_rep_red = nn.Linear(emb_sz, label_emb_sz)
lin_for_rep_red

Linear(in_features=400, out_features=49, bias=True)

In [ ]:
set_bg('yellow')
out = lin_for_rep_red(out)
out.shape

torch.Size([128, 1406, 49])

Let's also compress the representation length of the the last dimension in `after_first_matmul`:

In [ ]:
after_first_matmul.shape

torch.Size([128, 6594, 400])

In [ ]:
set_bg('yellow')
after_first_matmul = lin_for_rep_red(after_first_matmul)
after_first_matmul.shape

torch.Size([128, 6594, 49])

Now, let's compress the number of tokens in `out` from 1406 to 50.

In [ ]:
set_bg('yellow')
out = out.permute(0,2,1).contiguous()
out.shape

torch.Size([128, 49, 1406])

In [ ]:
set_bg('yellow')
lin_for_tok_red = nn.Linear(1406, 50)
lin_for_tok_red

Linear(in_features=1406, out_features=50, bias=True)

In [ ]:
set_bg('yellow')
out = lin_for_tok_red(out)
out.shape

torch.Size([128, 49, 50])

In [ ]:
set_bg('yellow')
out = out.permute(0,2,1).contiguous()
out.shape

torch.Size([128, 50, 49])

---

Now we are good to add `after_first_matmul` to our compressed memory (stored in `out`)

In [ ]:
set_bg('yellow')
print(f"{out.shape = }, {after_first_matmul.shape = }")
after_nonlinearity = torch.tanh(out[:, :, None] + after_first_matmul[:,None])
print(f"{after_nonlinearity.shape = }")

out.shape = torch.Size([128, 50, 49]), after_first_matmul.shape = torch.Size([128, 6594, 49])
after_nonlinearity.shape = torch.Size([128, 50, 6594, 49])


Note what each of the dimension of `after_nonlinearity` means:  
In each of the **128 documents**, we are focussing on the last **50 (compressed from 1406) tokens**. Each one of those tokens has a **hidden (compressed) represenation vector of length 49** for each of the **6594 labels**.

In [ ]:
after_nonlinearity.device

device(type='cpu')

---

In [ ]:
set_bg('yellow')
def init_param(*sz): print(f"{sz = }"); return nn.Parameter(torch.randn(sz)/math.sqrt(sz[0]))

In [ ]:
# init_param(3)

In [ ]:
# init_param(3,4)

`V` is for the second matrix multiply:

In [ ]:
set_bg('yellow')
V = init_param(label_emb_sz)
V.shape

sz = (49,)


torch.Size([49])

*Step 3 of mini-nn: Doing a second matrix multiply:*

In [ ]:
set_bg('yellow')
attn_wgts = (after_nonlinearity @ V)
attn_wgts.shape

torch.Size([128, 50, 6594])

In [ ]:
attn_wgts[:3, :7]

tensor([[[ 0.5042, -0.4067,  0.2481,  ...,  0.4743,  0.6855,  0.3966],
         [ 0.3884, -0.3913,  0.1412,  ...,  0.3579,  0.4673,  0.2750],
         [ 0.3630, -0.3703,  0.1130,  ...,  0.3201,  0.4854,  0.1989],
         ...,
         [ 0.5318, -0.3458,  0.2158,  ...,  0.4740,  0.6765,  0.3409],
         [ 0.0388, -0.7875, -0.2704,  ...,  0.0134,  0.2340, -0.1294],
         [ 0.2823, -0.5718, -0.0493,  ...,  0.2117,  0.4972,  0.1334]],

        [[ 0.1241, -0.7451, -0.1825,  ...,  0.0953,  0.2921, -0.0203],
         [ 0.2909, -0.5416,  0.0215,  ...,  0.2523,  0.4723,  0.1960],
         [ 0.1338, -0.5911, -0.0852,  ...,  0.2431,  0.3780,  0.0520],
         ...,
         [ 0.7050, -0.1777,  0.4014,  ...,  0.7176,  0.8624,  0.6345],
         [ 0.4314, -0.4561,  0.2013,  ...,  0.4348,  0.5315,  0.3565],
         [ 0.0490, -0.7727, -0.2656,  ..., -0.0220,  0.2699, -0.0791]],

        [[ 0.9006,  0.1438,  0.7388,  ...,  1.0048,  1.0604,  0.8983],
         [-0.1268, -0.8799, -0.4262,  ..., -0

Note that the 
- 0th axis is for the bs
- 1st axis is actual attention wgts
- 2nd axis is the labels

At this point what we have is the following:  
- In all the **128 documents** we have **50 (compressed from 1406) numbers** for each of the **6594 labels**.
- These 50 numbers are presumably the amount of *attention* we need to pay to   
those **50 tokens** each of which have a **49 (compressed) length** hidden represenation vector

Taking linear combination of the memory (`out`) based on the `attn_wgts` to get `ctx`:

In [ ]:
out.shape, attn_wgts.shape

(torch.Size([128, 50, 49]), torch.Size([128, 50, 6594]))

In [ ]:
# out[None,...].shape, attn_wgts[...,None].shape
out[:, :, None].shape, attn_wgts[..., None].shape

(torch.Size([128, 50, 1, 49]), torch.Size([128, 50, 6594, 1]))

In [ ]:
set_bg('yellow')
ctx = (out[:, :, None] * attn_wgts[..., None])
ctx.shape

torch.Size([128, 50, 6594, 49])

At this point the meaning of `ctx` is:  
For each of the **128 documents**, for each of the **50 (compressed from 1406)** tokens, for each of the **6594 labels** we have a **hidden compressed representation of length 49**.

In [ ]:
ctx[:2, :7, :, :9]

tensor([[[[ 1.7465e-03,  6.1149e-02, -2.2250e-01,  ..., -1.0547e-02, -8.3051e-02, -7.1283e-02],
          [-1.4087e-03, -4.9323e-02,  1.7947e-01,  ...,  8.5075e-03,  6.6989e-02,  5.7497e-02],
          [ 8.5931e-04,  3.0086e-02, -1.0947e-01,  ..., -5.1895e-03, -4.0863e-02, -3.5073e-02],
          ...,
          [ 1.6427e-03,  5.7516e-02, -2.0928e-01,  ..., -9.9206e-03, -7.8116e-02, -6.7048e-02],
          [ 2.3742e-03,  8.3127e-02, -3.0247e-01,  ..., -1.4338e-02, -1.1290e-01, -9.6903e-02],
          [ 1.3736e-03,  4.8094e-02, -1.7500e-01,  ..., -8.2956e-03, -6.5320e-02, -5.6065e-02]],

         [[ 1.9480e-01, -1.0085e-01, -6.0489e-02,  ..., -1.6415e-02,  7.1366e-02, -7.5854e-02],
          [-1.9623e-01,  1.0160e-01,  6.0934e-02,  ...,  1.6536e-02, -7.1891e-02,  7.6413e-02],
          [ 7.0810e-02, -3.6661e-02, -2.1989e-02,  ..., -5.9670e-03,  2.5942e-02, -2.7574e-02],
          ...,
          [ 1.7948e-01, -9.2922e-02, -5.5732e-02,  ..., -1.5124e-02,  6.5753e-02, -6.9889e-02],
        

In [ ]:
set_bg('yellow')
ctx = ctx.sum(1)
ctx.shape

torch.Size([128, 6594, 49])

Now, let's decompress the hidden representation length from 49 back to 400 by doing a non-linearity followed by matrix multiply:

In [ ]:
set_bg('yellow')
lin_for_rep_decompress = nn.Linear(label_emb_sz, emb_sz)
ctx = lin_for_rep_decompress(torch.relu(ctx))
ctx.shape

torch.Size([128, 6594, 400])

Let's recap what we have so far `ctx`:  
For each of the **128 documents** for each of the **6594 labels** we have a **represenation vector of lenth 400** (we obtained this by paying attention to specific parts of the memory).

Now this `ctx` will get concatenated with whatever the decoder was previously using without attention (i.e., the output of `masked_concat_pool`)

---

In [ ]:
out_concat_pool = torch.randn((128,1200))
out_concat_pool.shape

torch.Size([128, 1200])

In [ ]:
set_bg('yellow')
out_concat_pool = out_concat_pool[:, None]
out_concat_pool.shape

torch.Size([128, 1, 1200])

Think of `out_concat_pool` as the label-agnostic **1200 length represenation** of each of the **128 documents**.

In [ ]:
set_bg('yellow')
out_concat_pool = out_concat_pool.repeat(1, n_labels, 1)
out_concat_pool.shape

torch.Size([128, 6594, 1200])

In [ ]:
# ctx = ctx.permute(1,0,2).contiguous()
ctx.shape

torch.Size([128, 6594, 400])

Think of `ctx` as label-specific **400 length representation** of each of the **128 documents**.

In [ ]:
set_bg('yellow')
out_concat_pool = torch.cat((out_concat_pool, ctx), dim=-1)
out_concat_pool.shape

torch.Size([128, 6594, 1600])

At this point we can flattan the features before feeding it into the Linear-BatchNorm-Droput layers:

In [ ]:
t = torch.randn(128, 6594, 1600)
t.shape

torch.Size([128, 6594, 1600])

In [ ]:
t.view(-1).shape

torch.Size([1350451200])

In [ ]:
t.view(t.shape[0], -1).shape

torch.Size([128, 10550400])

In [ ]:
set_bg('yellow')
out_concat_pool = out_concat_pool.view(out_concat_pool.shape[0], -1)
out_concat_pool.shape

torch.Size([128, 10550400])

---

Let's see how to perform batchnorm on this:

First let's see an example

In [ ]:
m = nn.BatchNorm1d(100)
m

BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
input = torch.randn(20, 100)

In [ ]:
m(input).shape

torch.Size([20, 100])

In [ ]:
m = nn.BatchNorm1d(18)
m

BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
input = torch.randn(20, 18, 100)

In [ ]:
m(input).shape

torch.Size([20, 18, 100])

In [ ]:
m = nn.BatchNorm1d(n_out)
m

BatchNorm1d(6594, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
m(out_concat_pool).shape

torch.Size([128, 6594, 1600])

---

##### Practice some ufuncs in PyTorch and Numpy:

In [ ]:
np.repeat(3,4)

array([3, 3, 3, 3])

In [ ]:
arr = np.array([[1,2], [3,4]])
arr

array([[1, 2],
       [3, 4]])

In [ ]:
np.repeat(arr, 2)

array([1, 1, 2, 2, 3, 3, 4, 4])

In [ ]:
np.repeat(arr, 3, axis=1)

array([[1, 1, 1, 2, 2, 2],
       [3, 3, 3, 4, 4, 4]])

In [ ]:
np.repeat(arr, [1, 2], axis=0)

array([[1, 2],
       [3, 4],
       [3, 4]])

In [ ]:
a = np.array([0, 1, 2])
a, a.ndim, a.shape

(array([0, 1, 2]), 1, (3,))

In [ ]:
np.tile(a, 2)

array([0, 1, 2, 0, 1, 2])

In [ ]:
np.tile(a, (2,3))

array([[0, 1, 2, 0, 1, 2, 0, 1, 2],
       [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [ ]:
np.tile(a, (2,1,2))

array([[[0, 1, 2, 0, 1, 2]],

       [[0, 1, 2, 0, 1, 2]]])

In [ ]:
b = np.array([[1,2], [3, 4]])
b, b.shape, b.ndim

(array([[1, 2],
        [3, 4]]),
 (2, 2),
 2)

In [ ]:
np.tile(b,2)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [ ]:
np.tile(b, (2,1))

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

In [ ]:
np.tile(b, (2,3))

array([[1, 2, 1, 2, 1, 2],
       [3, 4, 3, 4, 3, 4],
       [1, 2, 1, 2, 1, 2],
       [3, 4, 3, 4, 3, 4]])

In [ ]:
c = np.array([1, 2, 3, 4])
c

array([1, 2, 3, 4])

In [ ]:
np.tile(c, (4,3))

array([[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
       [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
       [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
       [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]])

---

In [ ]:
t = torch.randint(0, 5, (3,6))
t, t.shape

(tensor([[1, 1, 4, 0, 3, 1],
         [3, 1, 3, 0, 3, 2],
         [3, 2, 1, 0, 1, 0]]),
 torch.Size([3, 6]))

In [ ]:
t = t[:, None]
t, t.shape

(tensor([[[1, 1, 4, 0, 3, 1]],
 
         [[3, 1, 3, 0, 3, 2]],
 
         [[3, 2, 1, 0, 1, 0]]]),
 torch.Size([3, 1, 6]))

In [ ]:
t.repeat(1, 5, 1)

tensor([[[1, 1, 4, 0, 3, 1],
         [1, 1, 4, 0, 3, 1],
         [1, 1, 4, 0, 3, 1],
         [1, 1, 4, 0, 3, 1],
         [1, 1, 4, 0, 3, 1]],

        [[3, 1, 3, 0, 3, 2],
         [3, 1, 3, 0, 3, 2],
         [3, 1, 3, 0, 3, 2],
         [3, 1, 3, 0, 3, 2],
         [3, 1, 3, 0, 3, 2]],

        [[3, 2, 1, 0, 1, 0],
         [3, 2, 1, 0, 1, 0],
         [3, 2, 1, 0, 1, 0],
         [3, 2, 1, 0, 1, 0],
         [3, 2, 1, 0, 1, 0]]])

---

In [ ]:
b = torch.tensor([[1], [2], [3]])
b.size()

torch.Size([3, 1])

In [ ]:
b.expand(3, 4)

tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

In [ ]:
b.expand(5,3)

RuntimeError: The expanded size of the tensor (5) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [5, 3].  Tensor sizes: [3, 1]

---

### `Learner` Creation: 

In this section we will replicate all the steps in `text_classifier_learner`:  

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.01, metrics=PrecisionK).to_fp16()

There are three things that happens inside `text_classifier_learner`
- create the `model` - we have already done this in the previous section
- create a `Learner` using `TextLearner`
    * `TextLearner` is a Basic class for a `Learner` in NLP.
    * Adds a `ModelResetter` and an `RNNRegularizer` with `alpha` and `beta` to the callbacks, the rest is same as `Learner` init.
    * Adding functionality to the base class, it has the methods - `load_pretrained`, `save_encoder` and `load_encoder`.
- load the pretrained weights (grabbing that information from `_model_meta`) into the encoder

In [ ]:
meta = _model_meta[arch]

#### Metrics: 

In [ ]:
def precision_at_k(yhat_raw, y, k=15):
    """
        Inputs: 
            yhat_raw: activation matrix of ndarray and shape (n_samples, n_labels)
            y: binary ground truth matrix of type ndarray and shape (n_samples, n_labels)
            k: for @k metric
    """
    yhat_raw, y = to_np(yhat_raw), to_np(y)
    # num true labels in the top k predictions / k
    sortd = yhat_raw.argsort()[:,::-1]
    topk = sortd[:, :k]
    
    # get precision at k for each sample
    vals = []
    for i, tk in enumerate(topk):
        num_true_in_top_k = y[i,tk].sum()
        vals.append(num_true_in_top_k / float(k))
    
    return np.mean(vals)

def precision_at_r(yhat_raw, y):
    """
        Inputs: 
            yhat_raw: activation matrix of ndarray and shape (n_samples, n_labels)
            y: binary ground truth matrix of type ndarray and shape (n_samples, n_labels)
    """
    yhat_raw, y = to_np(yhat_raw), to_np(y)
    # num true labels in the top r predictions / r, where r = number of labels associated with that sample 
    sortd = yhat_raw.argsort()[:, ::-1]
    
    # get precision at r for each sample
    vals = []
    for i, sorted_activation_indices in enumerate(sortd):
        # compute the number of labels associated with this sample
        r = int(y[i].sum())
        top_r_indices = sorted_activation_indices[:r] 
        num_true_in_top_r = y[i, top_r_indices].sum()
        vals.append(num_true_in_top_r / float(r))
    
    return np.mean(vals)

#### `TextLearner`

Let's create the `Learner` (we can pass our metrics here:)

In [ ]:
learn = TextLearner(dls_clas, model, splitter=meta['split_clas'], metrics=[precision_at_k, precision_at_r])

Note that in this case fastai was clever to figure out the correct loss (`BCEWithLogitsLoss`) from the `Dataloaders dls_clas`! We could overide this cleverness by specifically passing a `loss_func`.

In [ ]:
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

#### Pretrained Weights 

Now let's replace the random weights of the encoder using the pretrained weights (from the url in meta). Note that in this case we can skip this step, because we are anyway later on going to load the encoder weights from our finetuned language model. But let's not skip this step for completeness. 

In [ ]:
backwards = False
url = 'url_bwd' if backwards else 'url'

Let's check if we have pretrained weights for that `arch`:

In [ ]:
if url not in meta: warn("There are no pretrained weights for that architecture yet!")

Get the path which contains the pretrained model:

In [ ]:
pretrained_model_path = untar_data(meta[url], c_key='model')
pretrained_model_path.ls()

(#2) [Path('/home/ubuntu/.fastai/models/wt103-fwd/lstm_fwd.pth'),Path('/home/ubuntu/.fastai/models/wt103-fwd/itos_wt103.pkl')]

Let's get the weights of the pretrained model and the vocab that was used to train it:

In [ ]:
fnames = [list(pretrained_model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
wgts_fname, vocab_fname = fnames
wgts_fname, vocab_fname

(Path('/home/ubuntu/.fastai/models/wt103-fwd/lstm_fwd.pth'),
 Path('/home/ubuntu/.fastai/models/wt103-fwd/itos_wt103.pkl'))

Just for fun why don't we take a look at the pretrained weights and the vocab that got used to train that model:  

In [ ]:
pretrained_wgts, old_vocab = torch.load(wgts_fname), load_pickle(vocab_fname)
type(pretrained_wgts), type(old_vocab)

(collections.OrderedDict, list)

In [ ]:
for key in pretrained_wgts: print(key, end='  ')

0.encoder.weight  0.encoder_dp.emb.weight  0.rnns.0.weight_hh_l0_raw  0.rnns.0.module.weight_ih_l0  0.rnns.0.module.weight_hh_l0  0.rnns.0.module.bias_ih_l0  0.rnns.0.module.bias_hh_l0  0.rnns.1.weight_hh_l0_raw  0.rnns.1.module.weight_ih_l0  0.rnns.1.module.weight_hh_l0  0.rnns.1.module.bias_ih_l0  0.rnns.1.module.bias_hh_l0  0.rnns.2.weight_hh_l0_raw  0.rnns.2.module.weight_ih_l0  0.rnns.2.module.weight_hh_l0  0.rnns.2.module.bias_ih_l0  0.rnns.2.module.bias_hh_l0  1.decoder.weight  1.decoder.bias  

In [ ]:
print(coll_repr(old_vocab, 50))

(#60000) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxmaj','xxup','xxrep','xxwrep','the',',','.','of','and','in','to','a','=','"','was','on','-',"'s",'as','for','that','with','by','\n ',')','(','\n \n ','is','his','at','he','it','from','were','an','had','which','be','this','but',"'",'are','not','first','their'...]


Now we write that magic line which will replace the random weights in our `encoder` i.e., `learn.model[0]`, with the `pretrained_wgts`.  

**IMPORTANT**: We need to pass the
- `wgts_fname`,
- and the `vocab_fname`

In [ ]:
#learn.model[0]

In [ ]:
learn = learn.load_pretrained(wgts_fname, vocab_fname, model=learn.model[0])

Additionally, we also need to freeze the `encoder`:

In [ ]:
learn.freeze()

Also, to do mixed precision training, we need the following: 

In [ ]:
learn = learn.to_fp16()

#### LM weights:

Additionally, we also need to freeze the `encoder`:

In [ ]:
learn.freeze()

Also, to do mixed precision training, we need the following: 

In [ ]:
learn = learn.to_fp16()

**IMPORTANT:**  
This is where the "Magic" happens

The final step prior to training the classifier is to load the encoder from our fine-tuned language model. We will use `load_encoder` instead of `load` because we have only the pretrained weights available for the encoder; `load` by default raises an exception if an incomplete model is loaded:

In [ ]:
# !ls -lht {path_model}

In [ ]:
learn = learn.load_encoder(path_model/'lm_finetuned')